In [2]:
import sys
import logging
import time
import libtorrent as lt
from PyQt6.QtCore import QCoreApplication, QTimer
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QVBoxLayout, QWidget, QPushButton, QLabel, QFileDialog, QListWidget, QProgressBar
)

# Set up logging
logging.basicConfig(level=logging.DEBUG)

class TorrentMetadataClient(QMainWindow):
    def __init__(self):
        super().__init__()

        # Set up the UI
        self.init_ui()

        # Initialize libtorrent session
        self.session = lt.session()
        self.handle = None

    def init_ui(self):
        self.setWindowTitle("Torrent Metadata Viewer")
        self.setGeometry(100, 100, 600, 400)

        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        layout = QVBoxLayout(central_widget)

        self.open_torrent_button = QPushButton("Open Torrent File", self)
        self.open_torrent_button.clicked.connect(self.open_torrent)
        layout.addWidget(self.open_torrent_button)

        self.torrent_list = QListWidget(self)
        layout.addWidget(self.torrent_list)

        self.status_label = QLabel("Status: Idle", self)
        layout.addWidget(self.status_label)

        self.start_operation_button = QPushButton("Start Download", self)
        self.start_operation_button.clicked.connect(self.start_operation)
        layout.addWidget(self.start_operation_button)

        self.progress_bar = QProgressBar(self)
        layout.addWidget(self.progress_bar)

        # Timer to update the progress bar
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_progress)

    def open_torrent(self):
        torrent_file, _ = QFileDialog.getOpenFileName(self, "Open Torrent File", "", "Torrent Files (*.torrent)")
        logging.debug(f"Selected file: {torrent_file}")
        if not torrent_file:
            logging.debug("No file selected.")
            return

        # Parse the torrent file
        try:
            self.torrent_info = lt.torrent_info(torrent_file)
            logging.debug(f"Torrent loaded: {self.torrent_info.name()}")

            # Update the UI
            self.torrent_list.clear()
            self.torrent_list.addItem(f"Name: {self.torrent_info.name()}")
            self.torrent_list.addItem(f"Total Size: {self.torrent_info.total_size()} bytes")
            self.torrent_list.addItem(f"Files:")
            for file in self.torrent_info.files():
                self.torrent_list.addItem(f"  - {file.path} ({file.size} bytes)")

            self.status_label.setText(f"Loaded: {self.torrent_info.name()}")
        except Exception as e:
            logging.error(f"Error loading torrent: {e}")
            self.status_label.setText(f"Error: {str(e)}")

    def start_operation(self):
        if not hasattr(self, 'torrent_info'):
            self.status_label.setText("No torrent file loaded.")
            return

        # Add torrent to session
        params = {
            'save_path': './downloads/',
            'storage_mode': lt.storage_mode_t.storage_mode_sparse,
            'ti': self.torrent_info
        }
        self.handle = self.session.add_torrent(params)
        self.status_label.setText("Download started...")
        self.timer.start(1000)  # Update progress every second

    def update_progress(self):
        if self.handle is None:
            return

        s = self.handle.status()
        progress = s.progress * 100
        self.progress_bar.setValue(int(progress))
        self.status_label.setText(f"Downloading: {progress:.2f}%")

        if s.is_seeding:
            self.timer.stop()
            self.status_label.setText("Download completed.")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    client = TorrentMetadataClient()
    client.show()
    sys.exit(app.exec())

: 